### Optimization of n-D Rosenbrock function

Reference: https://en.wikipedia.org/wiki/Rosenbrock_function

$$ cost(a,b,\mathbf{x}) = \sum_{i=0}^{N/2+1} b (x_{2i+1}-x_{2i})^2 + (x_{2i}-a)^2 ,$$
$$pdf(a,b,\mathbf{x}) = e^{-cost(a,b,\mathbf{x})}$$ 

Here, $\mathbf{x}_{task}=(a,b)$ and $\mathbf{x}_{decision} = \mathbf{x}$

The global optima is uniquely given by $(a,a^2,a,a^2,\ldots, a,a^2)$

We show that TTGO is able to find the global optima consistently with a hand few of samples from the constructed tt-model of the above pdf (constructed offline) for various selection of $\mathbf{x}_{task}$ in the online phase. However, a naive approach of sampling from uniform distribution to initialize a Netwon-type optimizer does not work for larger $n$ (try $n=10$ in this notebook). We use scipy's SLSQP to fine tune the initialization provided by TTGO and uniform distribution.

Copyright (c) 2008 Idiap Research Institute, http://www.idiap.ch/
    Written by Suhan Shetty <suhan.shetty@idiap.ch>,


In [1]:
import torch
torch.set_default_dtype(torch.float64)

import numpy as np
import sys
sys.path.append('./fcn_opt')
sys.path.append('../')

from ttgo import TTGO
from test_fcns import Rosenbrock_nD 
from fcn_plotting_utils import plot_surf, plot_contour

%load_ext autoreload
np.set_printoptions(precision=2)
torch.set_printoptions(precision=2)

%autoreload 2

#### Define the cost and the correpsonding pdf function

In [2]:
n=20
pdf, cost =  Rosenbrock_nD(n=n,alpha=0.01) # n>=4

#### Define the domain and the discretization

In [3]:
# Define the domain of the function

L = 2 # [-L,L]^n is the domain of the function

# domain of task params: domain of coefficients a and b in Rosenbrock_4D
# Note: a should in (-sqrt(L), sqrt(L)) 
domain_task = [torch.linspace(-np.sqrt(L),np.sqrt(L),500)]+[torch.linspace(50,150,500)] 
# domain of decison varibales
domain_decision = [torch.linspace(-L,L,500)]*(n)
domain = domain_task+domain_decision

### Fit the TT-Model

In [ ]:
# Find the tt-model corresponding to the pdf
tt_model = tt_utils.cross_approximate(fcn=pdf,  domain=domain, 
                        rmax=200, nswp=20, eps=1e-3, verbose=True, 
                        kickrank=5, device=device)

In [4]:
ttgo = TTGO(domain=domain,tt_model=tt_model, cost=cost)

#### Specify task parameters

In [7]:
a = 0; b = 100;
x_task = torch.tensor([a,b]).to(device).view(1,-1) #given task-parameters

### Sample from TT-Model

In [10]:
n_samples_tt = 5

samples = ttgo.sample_tt(n_samples=n_samples_tt, x_task=x_task, alpha=0.5) 


### Choose the best sample as an estimate for optima (initial guess)

In [11]:
best_estimate = ttgo.choose_best_sample(samples)[0]
top_k_estimate = ttgo.choose_top_k_sample(samples,k=1)[0] # for multiple solutions

##### Fine-tune the estimate using gradient-based optimization

In [12]:
ttgo_optimized,_ = ttgo.optimize(best_estimate)

ttgo_optimized_k = 1*top_k_estimate
for i, x in enumerate(ttgo_optimized_k):
    ttgo_optimized_k[i],_ = ttgo.optimize(x)


In [13]:
print("PDF at the estimated point(initial guess): ", pdf(best_estimate))
print("PDF at the TTGO Optima: ", pdf(ttgo_optimized))

PDF at the estimated point(initial guess):  tensor([0.01])
PDF at the TTGO Optima:  tensor([1.00])


In [14]:
print("Estimated Optima: ", best_estimate)
print("Optima from TTGO: ", ttgo_optimized)

Estimated Optima:  tensor([[-2.83e-03,  9.99e+01, -7.66e-01,  9.10e-01,  1.11e+00,  9.74e-01,
          6.37e-01,  2.20e-01,  2.53e-01,  2.53e-01,  4.01e-03, -2.36e-01,
          5.21e-02, -5.17e-01,  3.61e-02,  7.90e-01, -6.61e-01,  8.14e-01,
         -2.69e-01, -2.00e-02,  8.42e-02,  3.09e-01]])
Optima from TTGO:  tensor([[-2.83e-03,  9.99e+01, -1.95e-02,  8.09e-04,  4.06e-04,  1.53e-04,
          1.97e-04, -3.36e-04, -7.23e-05, -3.59e-05, -5.69e-07, -1.52e-05,
         -9.01e-08, -1.04e-04, -2.93e-05, -5.60e-05, -4.27e-05,  7.92e-06,
          3.55e-05, -2.10e-05,  2.33e-05, -7.57e-05]])


#### Global Optima from Analytical Evaluation

In [15]:
global_optima = torch.tensor([ [a,b] +[a**(i%2+1) for i in range(n)]]).view(1,-1)
print("Global Optima (analytical): ", global_optima )
print("PDF at the gloabl optima: ", pdf(global_optima))

Global Optima (analytical):  tensor([[  0, 100,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0]])
PDF at the gloabl optima:  tensor([1.])
